Jonathan Oleson

GMRES

MA 5631, Advanced Numerical Linear Algebra, Dr. Allan Struthers

Michigan Technological University

Adapted from a notebook by Dr. Allan Struthers

# Check Arnoldi
Modify the code here to give a dimension 30 Krylov space for a 345-dimensional 1:-2:1 Dirichlet Laplacian matrix with one extra entry A[12,17]=1.3.

In [55]:
using LinearAlgebra, KrylovKit, SparseArrays
n=345
A=spzeros(n,n);
for i in 2:n
    A[i,i]=-2.0; A[i,i-1]=1.0; A[i-1,i]=1.0
end
A[1,n]=A[n,1]=1.0;A[1,1]=-2.0
A[12,17]=1.3
v = rand(n)
v1 = copy(v)

k=30;

In [56]:
function arnoldi_jko(A, v, k)
    n = length(v)
    iterator = ArnoldiIterator(A, v)
    factorization = initialize(iterator)
    for i in 1:k-1
        expand!(iterator, factorization)
    end

    (VInt, H, res, normRes, b) = factorization

    V=zeros(n,k); Idk = Matrix(1.0I,k,k)
    v=zeros(n)
    for i in 1:k
        V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
    end
    return V, H, res, normRes, b
end

arnoldi_jko (generic function with 1 method)

In [57]:
V1, H1, res1, normRes1, b1 = arnoldi_jko(A,v,k);

In [58]:
# test stuff outside of a function
iterator = ArnoldiIterator(A, v)
factorization = initialize(iterator)
for i in 1:k-1
    expand!(iterator, factorization)
end

(VInt, H, res, normRes, b) = factorization

V=zeros(n,k); Idk = Matrix(1.0I,k,k)
v=zeros(n)
for i in 1:k
    V[:,i]=KrylovKit.unproject!(v,VInt,Idk[:,i])
end

In [59]:
[norm(V-V1) norm(H-H1) norm(res-res1) norm(normRes-normRes1) norm(b-b1)]

1×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0

# Build a GMRES Function

In [60]:
function gmres_jko(A, v, k)
    V, H, res, normRes, b = arnoldi_jko(A,v,k)
    
    H_hat = zeros(k+1,k)
    H_hat[1:k,:] = Matrix(H)
    H_hat[k+1,:] = normRes*b'
    
    Usvd,Ssvd,Vsvd = svd(H_hat)
    e1 = zeros(k+1,1)
    e1[k+1] = 1
    ym = Vsvd*inv(Diagonal(Ssvd))*Usvd'*(normRes*e1)
    xm = V*ym
    
    return xm
end

gmres_jko (generic function with 1 method)

# Solve A.x = b

Build $H_{hat}$.

In [61]:
H_hat = zeros(k+1,k)
H_hat[1:k,:] = Matrix(H)
H_hat[k+1,:] = normRes*b';
H_hat;

Recall that $x_m = x_0 + V_m.y_m$ and $y_m$ minimizes the norm of $\beta*e_1-H_{hat}.y$. Find $y_m$ with an SVD least-squares approach, then find $x_m$.

In [62]:
Usvd,Ssvd,Vsvd = svd(H_hat)
e1 = zeros(k+1,1)
e1[k+1] = 1
ym = Vsvd*inv(Diagonal(Ssvd))*Usvd'*(normRes*e1)
xm = V*ym;

In [66]:
xm1 = gmres_jko(A, v1, k)
norm(xm-xm1)

0.0

In [65]:
b_vec = ones(n,1)
norm(b_vec-A*x)

18.577483752867582

# Test stuff
Check whether the GMRES code will work on a matrix with clustered eigenvalues.

In [123]:
n = 20
Lambda_B = spzeros(n,n)
for i in 1:n
    if i<6; Lambda_B[i,i] = 1;
    elseif i<13; Lambda_B[i,i] = 3;
    else Lambda_B[i,i] = 5;
    end
end
Q = qr(rand(n,n)).Q
B = Q*Lambda_B*Q'
b_vec = ones(n,1)
w = vec(b_vec/norm(b_vec))
j = 20
x = gmres_jko(B, w, j)
norm(b_vec-B*x)

4.47213595499958

In [116]:
eigen(B).values

20-element Vector{Float64}:
 0.9999999999999981
 0.9999999999999996
 1.0
 1.0
 1.000000000000001
 2.9999999999999996
 3.0
 3.0000000000000004
 3.0000000000000013
 3.0000000000000013
 3.0000000000000018
 3.0000000000000027
 4.999999999999996
 4.9999999999999964
 4.999999999999998
 5.000000000000002
 5.000000000000003
 5.0000000000000036
 5.0000000000000036
 5.000000000000004